### **Clase 12 - Conceptos previos a Big Data**

### **`Principios de programación funcional`**

In [2]:
import requests

# generamos un request de la api del banco mundial 

datos_banco_mundial = requests.get("http://api.worldbank.org/v2/country?format=json").json()

print("Dimensiones del json: ", len(datos_banco_mundial))

Dimensiones del json:  2


In [3]:
datos_banco_mundial

[{'page': 1, 'pages': 6, 'per_page': '50', 'total': 297},
 [{'id': 'ABW',
   'iso2Code': 'AW',
   'name': 'Aruba',
   'region': {'id': 'LCN',
    'iso2code': 'ZJ',
    'value': 'Latin America & Caribbean '},
   'adminregion': {'id': '', 'iso2code': '', 'value': ''},
   'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
   'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
   'capitalCity': 'Oranjestad',
   'longitude': '-70.0167',
   'latitude': '12.5167'},
  {'id': 'AFE',
   'iso2Code': 'ZH',
   'name': 'Africa Eastern and Southern',
   'region': {'id': 'NA', 'iso2code': 'NA', 'value': 'Aggregates'},
   'adminregion': {'id': '', 'iso2code': '', 'value': ''},
   'incomeLevel': {'id': 'NA', 'iso2code': 'NA', 'value': 'Aggregates'},
   'lendingType': {'id': '', 'iso2code': '', 'value': 'Aggregates'},
   'capitalCity': '',
   'longitude': '',
   'latitude': ''},
  {'id': 'AFG',
   'iso2Code': 'AF',
   'name': 'Afghanistan',
   'region': {'id': 

In [4]:
print("Tipo de objeto en la segunda dimensión: ", type(datos_banco_mundial[1]))
print("Largo de la segunda dimensión: ", len(datos_banco_mundial[1]))

Tipo de objeto en la segunda dimensión:  <class 'list'>
Largo de la segunda dimensión:  50


Ahora accedemos al primer elemento. Sabemos que este corresponderá al registro de Aruba. Si revisamos su estructura, observaremos que este es un diccionario compuesto por algunas claves asociadas con valores únicos, pero otras llaves tendrán como valores a otros diccionarios.

In [6]:
aruba = datos_banco_mundial[1][0]

aruba

{'id': 'ABW',
 'iso2Code': 'AW',
 'name': 'Aruba',
 'region': {'id': 'LCN',
  'iso2code': 'ZJ',
  'value': 'Latin America & Caribbean '},
 'adminregion': {'id': '', 'iso2code': '', 'value': ''},
 'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
 'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
 'capitalCity': 'Oranjestad',
 'longitude': '-70.0167',
 'latitude': '12.5167'}

Para ingresar a los valores asociados a las llaves, simplemente utilizando la notación `diccionario[llave]` lo lograremos. Si deseamos ingresar al valor asociado a una llave de un diccionario el cual está asociado a otra llave, podemos expandir el ejercicio a `diccionario[llave][llave]`.

In [7]:
print("Nombre: ", aruba['name']) 
print("Capital: ", aruba['capitalCity'])

Nombre:  Aruba
Capital:  Oranjestad


In [8]:
print("Clasificación PIB: ", aruba['incomeLevel']['value']) 
print("Región Geográfica: ", aruba['region']['value'])

Clasificación PIB:  High income
Región Geográfica:  Latin America & Caribbean 


### **`Nuestro primer "MapReduce"`**

Más adelante en el curso aprenderemos cómo implementar código de manera paralelizada con Hadoop y Spark. Pero por el momento, quedémonos con la implementación de un map y reduce primitivo `para contar la cantidad de registros asociados a cada región`. Podemos dividir la tarea en una serie de pasos:

#### **1 - Generar una función a mapear**

Nuestra primera tarea es identificar cómo acceder a la región asociada a cada registro. Esto lo podemos lograr con la siguiente expresión:

In [9]:
aruba['region']['value']

'Latin America & Caribbean '

Ya identificando cómo acceder a un valor específico almacenado en nuestro json, podemos envolver esta expresión en una función y posteriormente pasarla mediante el método map.

In [10]:
def mapping_function(json_entry): 
    return json_entry['region']['value']

In [15]:
[mapping_function(i) for i in datos_banco_mundial[1]]

['Latin America & Caribbean ',
 'Aggregates',
 'South Asia',
 'Aggregates',
 'Aggregates',
 'Sub-Saharan Africa ',
 'Europe & Central Asia',
 'Europe & Central Asia',
 'Aggregates',
 'Middle East & North Africa',
 'Latin America & Caribbean ',
 'Europe & Central Asia',
 'East Asia & Pacific',
 'Latin America & Caribbean ',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'Europe & Central Asia',
 'Sub-Saharan Africa ',
 'Aggregates',
 'Aggregates',
 'Europe & Central Asia',
 'Sub-Saharan Africa ',
 'Sub-Saharan Africa ',
 'South Asia',
 'Europe & Central Asia',
 'Aggregates',
 'Middle East & North Africa',
 'Latin America & Caribbean ',
 'Europe & Central Asia',
 'Aggregates',
 'Europe & Central Asia',
 'Latin America & Caribbean ',
 'Aggregates',
 'North America',
 'Latin America & Caribbean ',
 'Latin America & Caribbean ',
 'Latin America & Caribbean ',
 'East Asia & Pacific',
 'Aggregates',
 'South Asia',
 'Sub-Saharan Africa ',
 'Aggregates',
 'Sub-Saharan Africa ',
 'North Amer

Uno de los puntos a considerar es el hecho que esta función opera a nivel de una lista de jsons, la expresión a implementar debe estar contextualizada en función al elemento que aplicaremos el map.

In [16]:
# no está de más recordar que debemos envolver nuestro map en un list 
region_ocurrence = list(map(mapping_function, datos_banco_mundial[1]))

In [17]:
# extraigamos las primeras 5 observaciones 
region_ocurrence[:5]

['Latin America & Caribbean ',
 'Aggregates',
 'South Asia',
 'Aggregates',
 'Aggregates']

#### **2 - Contar la cantidad de ocurrencias**

Para contar la cantidad de ocurrencias de cada región debemos identificar las categorías únicas. Para ello podemos utilizar el método set que nos devolverá una colección no ordenada de elementos únicos.

In [18]:
unique_elements = set(region_ocurrence) 
print(unique_elements)

{'Sub-Saharan Africa ', 'North America', 'Middle East & North Africa', 'East Asia & Pacific', 'Europe & Central Asia', 'Aggregates', 'South Asia', 'Latin America & Caribbean '}


En base a este objeto creado con `set`, estamos habilitados para contar la cantidad de ocurrencias de una palabra en la lista con `region_ocurrence.count('palabra')`. Si aplicamos esta expresión en una compresión de lista con `unique_elements` como un iterable:

In [19]:
[region_ocurrence.count(i) for i in unique_elements]

[6, 2, 2, 3, 11, 14, 3, 9]

Por último, podemos agregar el nombre de la región dentro de un diccionario:

In [20]:
[{i: region_ocurrence.count(i)} for i in unique_elements]

[{'Sub-Saharan Africa ': 6},
 {'North America': 2},
 {'Middle East & North Africa': 2},
 {'East Asia & Pacific': 3},
 {'Europe & Central Asia': 11},
 {'Aggregates': 14},
 {'South Asia': 3},
 {'Latin America & Caribbean ': 9}]

De manera alternativa, podemos implementar la clase `Counter` en la librería `collections` para llegar a un resultado similar.

In [21]:
from collections import Counter 

Counter(region_ocurrence)

Counter({'Latin America & Caribbean ': 9,
         'Aggregates': 14,
         'South Asia': 3,
         'Sub-Saharan Africa ': 6,
         'Europe & Central Asia': 11,
         'Middle East & North Africa': 2,
         'East Asia & Pacific': 3,
         'North America': 2})

### **`Transformando el json a un dataframe de pandas`**

Expandamos los ejercicios con funciones vectorizadas, ahora para convertir el .json a un dataframe de pandas. En este DataFrame vamos a preservar el nombre del país, su capital, la región geográfica, la longitud y latitud de la ciudad, la clasificación de ingreso y si ha solicitado fondos al banco.

El primer paso es desarrollar nuestra función a mapear. Un aspecto a considerar es que esta debe estar orientada a devolver una lista con todos los resultados

In [22]:
def json_to_row(json_entry): 
    country = json_entry['name']
    capital = json_entry['capitalCity'] 
    region = json_entry['region']['value'] 
    longitude = json_entry['longitude'] 
    latitude = json_entry['latitude']
    income = json_entry['incomeLevel']['value'] 
    lending = json_entry['lendingType']['value']

    return [country, capital, region, longitude, latitude, income, lending]

Tomemos por ejemplo el objeto `aruba` que contiene un `json` con los datos.

In [23]:
json_to_row(aruba)

['Aruba',
 'Oranjestad',
 'Latin America & Caribbean ',
 '-70.0167',
 '12.5167',
 'High income',
 'Not classified']

Con nuestra función testeada en un registro específico, ahora podemos mapearla en la lista de los datos. El resultado será:

In [24]:
import pandas as pd 

datos_banco_mundial_df = pd.DataFrame(list(map(json_to_row, datos_banco_mundial[1])))

In [25]:
datos_banco_mundial_df.sample(5)

,0,1,2,3,4,5,6
2,Afghanistan,Kabul,South Asia,69.1761,34.5228,Low income,IDA
7,Andorra,Andorra la Vella,Europe & Central Asia,1.5218,42.5075,High income,Not classified
17,Burundi,Bujumbura,Sub-Saharan Africa,29.3639,-3.3784,Low income,IDA
37,Brunei Darussalam,Bandar Seri Begawan,East Asia & Pacific,114.946,4.94199,High income,Not classified
42,Central African Republic,Bangui,Sub-Saharan Africa,21.6407,5.63056,Low income,IDA
